In [3]:
import cv2 as cv
import numpy as np
import os
import sklearn
from sklearn.svm import SVC
from skimage.feature import hog,local_binary_pattern
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV ,StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

import pickle

In [4]:
MASK_PATH = './Dataset/mask/'
NO_MASK_PATH = './Dataset/no_mask/'

MASK_PATH_PREPROCESSED = './PreProcessed/mask/'
NO_MASK_PATH_PREPROCESSED = './PreProcessed/no_mask/'



In [5]:
mask_data = []
no_mask_data = []

for i in os.listdir(MASK_PATH_PREPROCESSED):
    path = os.path.join(MASK_PATH_PREPROCESSED,i)
    img = cv.imread(path,cv.IMREAD_GRAYSCALE)
    mask_data.append(img)
   
for i in os.listdir(NO_MASK_PATH_PREPROCESSED):
    path = os.path.join(NO_MASK_PATH_PREPROCESSED,i)
    img = cv.imread(path,cv.IMREAD_GRAYSCALE)
    no_mask_data.append(img)

In [ ]:
masked_features = []
no_masked_features = []

numPoints = 24
radius = 8

for i in mask_data:
    lbp = local_binary_pattern(i, numPoints,radius, method="uniform")
    hog_feature = hog(lbp, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    masked_features.append(hog_feature)
    
for i in no_mask_data:
    lbp = local_binary_pattern(i, numPoints,radius, method="uniform")
    hog_feature = hog(lbp, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    no_masked_features.append(hog_feature)

In [ ]:
df = pd.DataFrame(masked_features)
df['label'] = 1

df1 = pd.DataFrame(no_masked_features)
df1['label'] = 0

merged = pd.concat([df,df1])
merged

In [9]:
y = merged['label']
x = merged.drop(columns=['label'])

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
# svm = SVC(kernel='rbf',gamma=0.01, C=0.1)
# svm.fit(X_train,y_train)


# cross_validation = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# param_grid = {'C': [0.1, 1, 10, 100, 1000],  
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#               'kernel': ['rbf']}  
  
# grid = GridSearchCV(SVC(), param_grid, cv=cross_validation,refit = True, verbose = 3) 
  
# grid.fit(X_train, y_train) 


In [ ]:
# print(grid.best_params_)

In [11]:
classifier = SVC(C= 10, gamma= 0.01, kernel= 'rbf')
classifier.fit(X_train, y_train)

SVC(C=10, gamma=0.01)

In [12]:
# best_model = grid.best_estimator_
# y_pred = best_model.predict(X_test)

ytrain_pred = classifier.predict(X_train)
ytest_pred = classifier.predict(X_test)


# print(f"Best parameters found: {grid.best_params_}")
# print(f"Best cross-validation score: {grid.best_score_}")

print(classification_report(y_train, ytrain_pred))
print(classification_report(y_test, ytest_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       437
           1       1.00      1.00      1.00       427

    accuracy                           1.00       864
   macro avg       1.00      1.00      1.00       864
weighted avg       1.00      1.00      1.00       864

              precision    recall  f1-score   support

           0       0.93      0.96      0.95       205
           1       0.96      0.94      0.95       222

    accuracy                           0.95       427
   macro avg       0.95      0.95      0.95       427
weighted avg       0.95      0.95      0.95       427



In [13]:

matrix_kebingungan = confusion_matrix(ytest_pred, y_test)
print(matrix_kebingungan)

matrix_kebingungan = confusion_matrix(ytrain_pred, y_train)
print(matrix_kebingungan)

[[197  14]
 [  8 208]]
[[437   0]
 [  0 427]]


In [14]:




# img = cv.imread('./b.jpg', cv.IMREAD_GRAYSCALE)
# # img = no_mask_data[-1:]
# # print(img)
# faces = haarcasade.detectMultiScale(img, scaleFactor = 1.2, minNeighbors = 5)
# if not len(faces) < 1:
#     for face_rect in faces:
#         x, y, w, h = face_rect
#         face_image = img[y:y+w, x:x+h]
#         resized = cv.resize(face_image,(128,128))

#         hog_feature = hog(resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
#         test_data = pd.DataFrame(hog_feature).T

#         result=best_model.predict(test_data)[0]
#         print(result)
#         if result == 0:
#             result = 'No Mask'
#         else:
#             result = 'Mask'


#         cv.putText(img, result, (100,100), cv.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 1)
#         cv.imshow('',img)
#         cv.waitKey(0)
#         cv.destroyAllWindows()

#         cv.imshow('',resized)
#         cv.waitKey(0)
#         cv.destroyAllWindows()

NameError: name 'haarcasade' is not defined

In [ ]:
import pickle
model_filename = './model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f"Model saved to {model_filename}")